# HW5: MapReduce

While mapreduce is an important concept in large scale parallel systems it is possible to simulate it locally and on small practice problems.  To this end we provide a simple framework that allows you to run and test map/reduce functions below.  This assignment asks you to write the code for two, two-stage mapreduce problems.

In [1]:
from collections import defaultdict
import string
import numpy as np

def simple_mapreduce(map_fn, reduce_fn, kvin):
    keyvalues = defaultdict(list)
    for k, v in kvin:
        for outk, outv in map_fn(k, v):
            keyvalues[outk] += [outv]
    kvout = []
    for k, vs in keyvalues.items():
        for outk, outv in reduce_fn(k, vs):
            kvout += [(outk, outv)]
    return kvout

## Example data
Some small speeches that are provided as an example dataset to play with 

In [2]:
speech_dataset = [
    ("emmeline pankhurst", """You have left it to women in your land, the men of all civilised countries have left it to women, to work out their own salvation. That is the way in which we women of England are doing. Human life for us is sacred, but we say if any life is to be sacrificed it shall be ours; we won’t do it ourselves, but we will put the enemy in the position where they will have to choose between giving us freedom or giving us death."""),
    ("gettysburg address", """Four score and seven years ago our fathers brought forth on this continent, a new nation, conceived in Liberty, and dedicated to the proposition that all men are created equal. Now we are engaged in a great civil war, testing whether that nation, or any nation so conceived and so dedicated, can long endure. We are met on a great battle-field of that war. We have come to dedicate a portion of that field, as a final resting place for those who here gave their lives that that nation might live. It is altogether fitting and proper that we should do this.  But, in a larger sense, we can not dedicate -- we can not consecrate -- we can not hallow -- this ground. The brave men, living and dead, who struggled here, have consecrated it, far above our poor power to add or detract. The world will little note, nor long remember what we say here, but it can never forget what they did here. It is for us the living, rather, to be dedicated here to the unfinished work which they who fought here have thus far so nobly advanced. It is rather for us to be here dedicated to the great task remaining before us -- that from these honored dead we take increased devotion to that cause for which they gave the last full measure of devotion -- that we here highly resolve that these dead shall not have died in vain -- that this nation, under God, shall have a new birth of freedom -- and that government of the people, by the people, for the people, shall not perish from the earth."""),
    ("churchill address", """I have, myself, full confidence that if all do their duty, if nothing is neglected, and if the best arrangements are made, as they are being made, we shall prove ourselves once again able to defend our Island home, to ride out the storm of war, and to outlive the menace of tyranny, if necessary for years, if necessary alone. At any rate, that is what we are going to try to do. That is the resolve of His Majesty's Government-every man of them. That is the will of Parliament and the nation. The British Empire and the French Republic, linked together in their cause and in their need, will defend to the death their native soil, aiding each other like good comrades to the utmost of their strength. Even though large tracts of Europe and many old and famous States have fallen or may fall into the grip of the Gestapo and all the odious apparatus of Nazi rule, we shall not flag or fail. We shall go on to the end, we shall fight in France, we shall fight on the seas and oceans, we shall fight with growing confidence and growing strength in the air, we shall defend our Island, whatever the cost may be, we shall fight on the beaches, we shall fight on the landing grounds, we shall fight in the fields and in the streets, we shall fight in the hills; we shall never surrender, and even if, which I do not for a moment believe, this Island or a large part of it were subjugated and starving, then our Empire beyond the seas, armed and guarded by the British Fleet, would carry on the struggle, until, in God's good time, the New World, with all its power and might, steps forth to the rescue and the liberation of the old."""),
    ("mlk dream", """    I have a dream that one day down in Alabama, with its vicious racists, with its governor having his lips dripping with the words of interposition and nullification – one day right there in Alabama little black boys and black girls will be able to join hands with little white boys and white girls as sisters and brothers.    I have a dream today.  I have a dream that one day every valley shall be exalted and every hill and mountain shall be made low, the rough places will be made plain, and the crooked places will be made straight, and the glory of the Lord shall be revealed and all flesh shall see it together."""),
    ("hrc human rights", """‘If there is one message that echoes forth from this conference, let it be that human rights are women’s rights and women’s rights are human rights once and for all. Let us not forget that among those rights are the right to speak freely — and the right to be heard.’""")
]

## Example usage

This simple example takes the above data and counts the occurrence of words using a mapreduce algorithm
Notice the use of yield instead of return.  yield is similar to return but for this purpose it allows us to return more than one time.

In [3]:
#k = title, v = text of speech
def ex1_map1(k, v):
    words = v.lower().split()  # simple tokenization, split by whitespaces
    for word in words:
        yield word, 1
    
# k = bigram, values = 1 for each bigram
def ex1_reduce1(k, values):
    yield k, sum(values)
    
def simple_example(dataset):
    return sorted(simple_mapreduce(ex1_map1, ex1_reduce1, dataset), key=lambda x: -x[1])

simple_example(speech_dataset)[:10]

[('the', 51),
 ('and', 34),
 ('we', 27),
 ('to', 25),
 ('that', 23),
 ('of', 20),
 ('shall', 20),
 ('in', 17),
 ('have', 12),
 ('be', 12)]

## Q1 Bigram statistics (25 pts):


Bigrams are two symbols that occur adjacently.  In this case the symbols are simply letters.  Below is a function `make_bigrams` that will turn a length of text into a list of bigrams.  Your job is to create a 2 step mapreduce function that first calculates the occurrence of bigrams and then in a second step calculates the average occurrence of bigrams that start with a particular letter.  Replace the `pass`'s with your code.  You may not edit any other code at this stage.

In [4]:
# you may use this function that converts a string of text into bigrams THE -> th he
def make_bigrams(text):
    just_txt = [c.lower() for c in text if c in string.ascii_letters]
    bigrams = [a + b for a,b in zip(just_txt[:-1], just_txt[1:])]
    return bigrams

In [5]:
#k = title, v = text
def q1_map1(k, v):
    bigrams = make_bigrams(v)
    for bigram in bigrams:
        yield bigram, 1

# k = bigram, values = 1 for each bigram
def q1_reduce1(k, values):
     yield k, sum(values)

# k = bigram, v the count for each bigram    
def q1_map2(k, v):
    # loop through every bigram, and convert it into '(letter), *)
    yield k[0]+"*", v
    
# k = the first letter of the bigram, v = a list of the # of bigrams that start with k  
def q1_reduce2(k, values):
    yield k, sum(values)/len(values)

def bigram_distribution(dataset):
    first_stage = simple_mapreduce(q1_map1, q1_reduce1, dataset)  
    bigram_final = simple_mapreduce(q1_map2, q1_reduce2, first_stage)
    return bigram_final

bigram_distribution(speech_dataset)

[('y*', 2.7058823529411766),
 ('o*', 10.826086956521738),
 ('u*', 6.090909090909091),
 ('h*', 17.923076923076923),
 ('a*', 14.95),
 ('v*', 14.25),
 ('e*', 19.82608695652174),
 ('l*', 9.0),
 ('f*', 6.5),
 ('t*', 17.05),
 ('i*', 13.166666666666666),
 ('w*', 9.555555555555555),
 ('m*', 6.777777777777778),
 ('n*', 11.6),
 ('r*', 9.952380952380953),
 ('d*', 8.105263157894736),
 ('c*', 7.2),
 ('s*', 8.428571428571429),
 ('k*', 1.4285714285714286),
 ('g*', 5.9375),
 ('b*', 5.333333333333333),
 ('p*', 4.222222222222222),
 ('q*', 1.0),
 ('j*', 1.0),
 ('z*', 1.0),
 ('x*', 1.0)]

In [6]:
bigram_expected = [
 ('y*', 2.7058823529411766),
 ('o*', 10.826086956521738),
 ('u*', 6.090909090909091),
 ('h*', 17.923076923076923),
 ('a*', 14.95),
 ('v*', 14.25),
 ('e*', 19.82608695652174),
 ('l*', 9.0),
 ('f*', 6.5),
 ('t*', 17.05),
 ('i*', 13.166666666666666),
 ('w*', 9.555555555555555),
 ('m*', 6.777777777777778),
 ('n*', 11.6),
 ('r*', 9.952380952380953),
 ('d*', 8.105263157894736),
 ('c*', 7.2),
 ('s*', 8.428571428571429),
 ('k*', 1.4285714285714286),
 ('g*', 5.9375),
 ('b*', 5.333333333333333),
 ('p*', 4.222222222222222),
 ('q*', 1.0),
 ('j*', 1.0),
 ('z*', 1.0),
 ('x*', 1.0)
]

assert sorted(bigram_expected) == sorted(bigram_distribution(speech_dataset))

---
## Q2: Network Degree distribution (25pts) 



The input given to you describes a dataset of nodes connected to each other via edges. An example of such a network could be the Twitter follower network, in which the users denote nodes and two nodes are connected by an edge if one follows the other.

 * The **degree** of a node in a network is the number of connections it has to other nodes. 
 * The **degree distribution** is the distribution of these degrees over the whole network.

As an example, you can assume that the input you are given is an edgelist in the form of a table as shown below. This table shows pairs of edges connected to each other and thus capture the graph shown on the right:

Source | Destination 
-------|-------------
 A | B
 A | C
 A | D 
 B | C

 
 The degree distribution then is 
 
 Degree | Count
 -------|-------
 1      | 1
 2      | 2
 3      | 1
 
 As you can see, we have one nodes with degree 1 (which is D), two node with degree 2 (which is B and C), and one node with degree 3 (which is A). Thus, the degree distribution captures, for each value of degree, the number of nodes that have that degree.  Below you can see the network mapped out

In [7]:
%matplotlib inline 
import matplotlib.pyplot as plt
 
plt.axis("off")
plt.scatter( [0, 0, 1, 0], [1, 0, 0, -1], s=1000, marker='o')
# Annotate with text + Arrow
plt.plot([0, 0], [-1, 1], c="r", zorder=-1)
plt.plot([0, 1], [0, 0],  c="r", zorder=-1)
plt.plot([0, 1], [-1, 0], c="r", zorder=-1)


plt.annotate('A', xy=(0, 0), xytext=(0, 0), c="w")
plt.annotate('B', xy=(1, 0), xytext=(1, 0), c="w")
plt.annotate('C', xy=(0, -1), xytext=(0, -1), c="w")
plt.annotate('D', xy=(0, 1), xytext=(0, 1), c="w");


In [8]:
network1_dataset = [
    ('A', 'B'),
    ('A', 'C'),
    ('A', 'D'),
    ('B', 'C'),
]

network1_expected = [
    (3, 1),
    (2, 2),
    (1, 1)
]

For Q2: you are to write the body of the following functions: (replace the `pass`'s with your code).  Do not change the `network_final` function.

In [9]:
# k=source node, v=destination node
def q2_map1(k, v):
    yield k, 1
    yield v, 1
    
def q2_reduce1(k, values):
    yield k, sum(values)

def q2_map2(k, v):
    yield v, 1

def q2_reduce2(k, values):
    yield k, sum(values)

def network_final(dataset):
    first_stage = simple_mapreduce(q2_map1, q2_reduce1, dataset)
    final = simple_mapreduce(q2_map2, q2_reduce2, first_stage)
    return final


network_final(network1_dataset)

[(3, 1), (2, 2), (1, 1)]

In [10]:
assert sorted(network1_expected) == sorted(network_final(network1_dataset))

In [11]:
network2_dataset = [(0, 1), (0, 2), (0, 3), (0, 5), (0, 6), (0, 9), (1, 3), (1, 5), (1, 6), (1, 7), (1, 8), (2, 3), (2, 4), (2, 5), (2, 7), (2, 8), (2, 9), (3, 5), (3, 6), (3, 7), (4, 5), (4, 7), (4, 8), (5, 6), (5, 7), (6, 7), (6, 8), (6, 9), (7, 8), (7, 9), (8, 9)]
network2_histogram = [0, 0, 0, 0, 1, 1, 4, 3, 1]
network2_expected = sorted([(deg, amt) for (deg, amt) in zip(range(9), network2_histogram) if amt > 0])
network2_expected

[(4, 1), (5, 1), (6, 4), (7, 3), (8, 1)]

In [12]:
assert network2_expected == sorted(network_final(network2_dataset))

## Q3 TFIDF Calculation  (50pts)

Partially from  the [sklearn docs](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer):

  Tf means term-frequency while tf-idf means term-frequency times inverse document-frequency. This is a common term weighting scheme in information retrieval, that has also found good use in document classification.

  The goal of using tf-idf instead of the raw frequencies of occurrence of a token in a given document is to scale down the impact of tokens that occur very frequently in a given corpus and that are hence empirically less informative than features that occur in a small fraction of the training corpus.

  The formula that is used to compute the tf-idf for a term t of a document d in a document set is $tfidf(t, d) = tf(t, d) * idf(t)$, and the idf is computed as $$idf(t) = \log{\frac{n}{df(t)}} + 1$$, where $n$ is the total number of documents in the document set and $df(t)$ is the document frequency of t; the document frequency is the number of documents in the document set that contain the term t. The effect of adding “1” to the idf in the equation above is that terms with zero idf, i.e., terms that occur in all documents in a training set, will not be entirely ignored.  Term frequency is simply the number of times the word appears in the document.

You will need to modify the map/reduce functions and provide the calculations for the appropriate portions of the `tfidf_final` function


In [13]:
from collections import Counter
import math

#useful functions might include:
def tokenize(doc):
    words_with_puncs = doc.lower().split()
    words = []
    for word in words_with_puncs:
        words += ["".join(char for char in word if str.isalpha(char))]
    return [w for w in words if w]

set()

def count_tokens(tokenized_doc):
    return list(Counter(tokenized_doc).items())
    #return [(word, counts) for word, counts in Counter(tokenized_doc)]       

In [14]:
# k=doc #, v=doc
def q3_idf_map(k, v):
    for token in tokenize(v):
        yield token, k

#k=word, v=1 for each doc containing word
def q3_idf_reduce(k, values):
    yield k, len(set(values))

# k = doc #, v=doc
# return (doc #, word), count of word
def q3_counts_map(k, v):
    for word, count in count_tokens(tokenize(v)):
        yield (k, word), count
    
#k = (doc #, word), v = count
# return k=(doc #, word), v = freq
def q3_counts_reduce(k, values):
    yield k, sum(values)

def tfidf_final(dataset):
    num_docs = len(dataset)
    dfs = simple_mapreduce(q3_idf_map, q3_idf_reduce, dataset)
    idfs = {}
    for word, df in dfs:
        idfs[word] =  math.log(num_docs/df) + 1# calculate idf per the comments above
    
    counts_per_doc = dict(simple_mapreduce(q3_counts_map, q3_counts_reduce, dataset))

    tfidf = {}
    for doc, word in counts_per_doc:
        freq = counts_per_doc[doc, word]
        tfidf[doc, word] = freq * idfs[word]
    return tfidf

tfidfs = tfidf_final(speech_dataset)
tfidfs

{('emmeline pankhurst', 'you'): 2.6094379124341005,
 ('emmeline pankhurst', 'have'): 3.669430653942629,
 ('emmeline pankhurst', 'left'): 5.218875824868201,
 ('emmeline pankhurst', 'it'): 4.0,
 ('emmeline pankhurst', 'to'): 5.0,
 ('emmeline pankhurst', 'women'): 7.8283137373023015,
 ('emmeline pankhurst', 'in'): 3.669430653942629,
 ('emmeline pankhurst', 'your'): 2.6094379124341005,
 ('emmeline pankhurst', 'land'): 2.6094379124341005,
 ('emmeline pankhurst', 'the'): 4.0,
 ('emmeline pankhurst', 'men'): 1.916290731874155,
 ('emmeline pankhurst', 'of'): 2.4462871026284194,
 ('emmeline pankhurst', 'all'): 1.0,
 ('emmeline pankhurst', 'civilised'): 2.6094379124341005,
 ('emmeline pankhurst', 'countries'): 2.6094379124341005,
 ('emmeline pankhurst', 'work'): 1.916290731874155,
 ('emmeline pankhurst', 'out'): 1.916290731874155,
 ('emmeline pankhurst', 'their'): 1.5108256237659907,
 ('emmeline pankhurst', 'own'): 2.6094379124341005,
 ('emmeline pankhurst', 'salvation'): 2.6094379124341005,
 ('

### Test case for the above

We choose twenty example cases to check the data against

In [15]:
some_expecteds = [(('gettysburg address', 'note'), 2.6094379124341005),
 (('emmeline pankhurst', 'ourselves'), 1.916290731874155),
 (('churchill address', 'or'), 4.532476871297972),
 (('churchill address', 'alone'), 2.6094379124341005),
 (('emmeline pankhurst', 'we'), 6.043302495063963),
 (('churchill address', 'old'), 5.218875824868201),
 (('gettysburg address', 'brave'), 2.6094379124341005),
 (('hrc human rights', 'those'), 1.916290731874155),
 (('gettysburg address', 'a'), 10.575779366361935),
 (('gettysburg address', 'proper'), 2.6094379124341005),
 (('gettysburg address', 'government'), 2.6094379124341005),
 (('churchill address', 'in'), 9.785148410513678),
 (('gettysburg address', 'we'), 15.108256237659907),
 (('gettysburg address', 'nor'), 2.6094379124341005),
 (('churchill address', 'neglected'), 2.6094379124341005),
 (('hrc human rights', 'there'), 1.916290731874155),
 (('mlk dream', 'mountain'), 2.6094379124341005),
 (('gettysburg address', 'advanced'), 2.6094379124341005),
 (('gettysburg address', 'resolve'), 1.916290731874155),
 (('churchill address', 'starving'), 2.6094379124341005)]

final_speech = tfidf_final(speech_dataset)
for k, v in some_expecteds:
    assert abs(final_speech[k] - v) < 1e-5